### Geo Dataset - Assignment

#### Step1 : Data Cleaning

In [1]:
#Importing Libraries
import pandas as pd
import numpy as np
import os 

In [2]:
# Set Working Directory
work_dir = %cd ~/geo_unlocode

C:\Users\avi140\geo_unlocode


In [3]:
# Listing Work Dir to Check if all the required files are present
os.listdir(work_dir) 

['.DS_Store',
 'code-list.csv',
 'country.txt',
 'function_list.txt',
 'Geo_Assignment.ipynb']

#### Load Country Data Set and Perform Basic Data Sanity Check


In [4]:
# Reasing Country File 
countryDF = pd.read_csv('country.txt',sep=',')

In [5]:
# top Five rows
countryDF.head()

,CountryCode,CountryName
0,AD,Andorra
1,AE,United Arab Emirates
2,AF,Afghanistan
3,AG,Antigua and Barbuda
4,AI,Anguilla


In [6]:
#shape of Pandas Dataframe
countryDF.shape

(249, 2)

In [7]:
#Basic info about the dataframe
countryDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 2 columns):
CountryCode    248 non-null object
CountryName    249 non-null object
dtypes: object(2)
memory usage: 4.0+ KB


In [8]:
#Basic data Sanity Checks
#Checking for Duplicates 
countryDF_Dups = countryDF[countryDF.duplicated()]

In [9]:
# No Dups at Row Level
countryDF_Dups.count()

CountryCode    0
CountryName    0
dtype: int64

In [10]:
countryCD_Dups = countryDF[countryDF.duplicated(["CountryCode"])]

In [11]:
countryCD_Dups

,CountryCode,CountryName


In [12]:
countryName_Dups = countryDF[countryDF.duplicated(["CountryName"])]

In [13]:
countryName_Dups

,CountryCode,CountryName


In [14]:
CountryCode_nan_rows = countryDF[countryDF['CountryCode'].isnull()]

In [15]:
CountryCode_nan_rows

,CountryCode,CountryName
149,NaN,Namibia


In [16]:
CountryName_nan_rows = countryDF[countryDF['CountryName'].isnull()]

In [17]:
CountryName_nan_rows

,CountryCode,CountryName


#### Load Function Or Location Type Data Set and Perform Basic Data Sanity Check

In [18]:
# Reading Country File 
functionDF = pd.read_csv('function_list.txt',sep=',')

In [19]:
# top Five rows
functionDF.head()

,FunctionCode,FunctionDescription
0,0,"Function not known, to be specified"
1,1,"Port, as defined in Rec 16"
2,2,Rail Terminal
3,3,Road Terminal
4,4,Airport


In [20]:
functionDF.shape

(10, 2)

In [21]:
functionDF.head(10)

,FunctionCode,FunctionDescription
0,0,"Function not known, to be specified"
1,1,"Port, as defined in Rec 16"
2,2,Rail Terminal
3,3,Road Terminal
4,4,Airport
5,5,Postal Exchange Office
6,6,"Multimodal Functions (ICDs, etc.)"
7,7,Fixed Transport Functions (e.g. Oil platform)
8,8,Inland Port
9,B,Border Crossing


###### Function Code 'B' Seems Ambiguous -- Lets See in the main Dataframe what all values of function Code is present

In [22]:
# Reading Code-list File 
mainDF = pd.read_csv('code-list.csv',sep=',')

In [23]:
mainDF.head()

,Change,Country,Location,Name,NameWoDiacritics,Subdivision,Status,Function,Date,IATA,Coordinates,Remarks
0,NaN,AU,ALO,Adelong,Adelong,NSW,RL,--3-----,607.0,NaN,3519S 14804E,NaN
1,NaN,AU,ANB,Agnes Bank,Agnes Bank,NSW,RL,--3-----,1501.0,NaN,3336S 15042E,NaN
2,NaN,AU,AGW,Agnew,Agnew,QLD,AI,---4----,9601.0,NaN,NaN,NaN
3,NaN,AU,WSY,Airlie Beach,Airlie Beach,QLD,AI,---4----,9912.0,NaN,NaN,NaN
4,NaN,AU,AIR,Airlie Terminal,Airlie Terminal,WA,RL,1-------,212.0,NaN,2120S 11510E,Airlie Island


In [24]:
mainDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109801 entries, 0 to 109800
Data columns (total 12 columns):
Change              2114 non-null object
Country             109756 non-null object
Location            109801 non-null object
Name                109801 non-null object
NameWoDiacritics    109801 non-null object
Subdivision         95028 non-null object
Status              109797 non-null object
Function            109801 non-null object
Date                109772 non-null float64
IATA                798 non-null object
Coordinates         83542 non-null object
Remarks             988 non-null object
dtypes: float64(1), object(11)
memory usage: 10.1+ MB


##### Note: mainDF needs to be cleansed before merging it with function_list and Country dataframe.     
Apart from The change column, which seems to have most NaN Values, If you closely look, the funtion coulmn is supposed have function code without additional dashes '-'     Approaching to clean this by implementing a regex to remove special characters. and thenreanalying distribution of function coulmn.

In [25]:
# Let's analyze distribution of Chnage Column

In [26]:
# Using reset index to convert it from Series to DF
mainDF.groupby('Change').size().reset_index(name='counts') 

,Change,counts
0,!,94
1,#,98
2,+,1394
3,X,90
4,¦,438


Change Column Doesn't make sense to the dataset, so we can drop this column

In [27]:
mainDF = mainDF.drop(['Change'], axis=1)

###### Looking at the Basinc info in athe above cell for mainDF, we see around 45 Rows having NaN values in Country Column. We can drop such rows.

In [28]:
mainDF = mainDF.dropna(subset=['Country'])

In [29]:
mainDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109756 entries, 0 to 109800
Data columns (total 11 columns):
Country             109756 non-null object
Location            109756 non-null object
Name                109756 non-null object
NameWoDiacritics    109756 non-null object
Subdivision         95015 non-null object
Status              109752 non-null object
Function            109756 non-null object
Date                109727 non-null float64
IATA                798 non-null object
Coordinates         83526 non-null object
Remarks             988 non-null object
dtypes: float64(1), object(10)
memory usage: 10.0+ MB


###### Let's target function column and it's distribution after removing special characters form the column.

In [30]:
mainDF['Function'] = mainDF['Function'].replace(to_replace=r'[^A-Za-z0-9]+', value='', regex=True)

In [31]:
mainDF.head()

,Country,Location,Name,NameWoDiacritics,Subdivision,Status,Function,Date,IATA,Coordinates,Remarks
0,AU,ALO,Adelong,Adelong,NSW,RL,3,607.0,NaN,3519S 14804E,NaN
1,AU,ANB,Agnes Bank,Agnes Bank,NSW,RL,3,1501.0,NaN,3336S 15042E,NaN
2,AU,AGW,Agnew,Agnew,QLD,AI,4,9601.0,NaN,NaN,NaN
3,AU,WSY,Airlie Beach,Airlie Beach,QLD,AI,4,9912.0,NaN,NaN,NaN
4,AU,AIR,Airlie Terminal,Airlie Terminal,WA,RL,1,212.0,NaN,2120S 11510E,Airlie Island


In [32]:
mainDF[mainDF['Function']== '1234567']

,Country,Location,Name,NameWoDiacritics,Subdivision,Status,Function,Date,IATA,Coordinates,Remarks
70200,KR,PUS,Busan,Busan,26,AF,1234567,1001.0,NaN,3508N 12903E,NaN


In [33]:
#mainDF[mainDF['Function'].apply(lambda x: len(x)>1)]
#function_TestDF = mainDF['Function'].apply(lambda x: len(x)>1)

In [34]:
# testing Function Column which has Function code length > 1
Function_TestDF = mainDF.loc[mainDF.Function.apply(lambda x: len(x)>1)]

In [35]:
# Checking destribution of Function Column in Function Test DF
Function_TestDF.groupby('Function').size().reset_index(name='counts').head(10)

,Function,counts
0,12,87
1,123,1777
2,1234,166
3,12345,78
4,123456,5
5,1234567,1
6,123457,1
7,12345B,1
8,12346,16
9,1234B,3


In [36]:
Function_TestDF.groupby('Function').size().reset_index(name='counts').sum()

Function    12123123412345123456123456712345712345B1234612...
counts                                                  24790
dtype: object

##### Note: There are 91 Such Combinations in around 25K rows
Now, Let's test what  is the distribution of Function Code where str length = 1


In [37]:
#function_TestDF = mainDF['Function'].apply(lambda x: len(x) = 1)
#functionLength1_TestDF = mainDF['Function'].apply(lambda x: True if len(x) == 1 else False)
functionLength1_TestDF = mainDF.loc[mainDF['Function'].apply(lambda x: len(x) == 1)]

In [38]:
# Checking destribution of Function Column in Function Test DF
functionLength1_TestDF.groupby('Function').size().reset_index(name='counts')

,Function,counts
0,0,882
1,1,7953
2,2,481
3,3,62235
4,4,5580
5,5,106
6,6,7457
7,7,198
8,B,74


In [39]:
functionLength1_TestDF.groupby('Function').size().reset_index(name='counts').sum()

Function    01234567B
counts          84966
dtype: object

###### Looking at the above result set, it seems that around 85003 rows have appropriate function code out of 109801, which is close to  78%

##### Creating Cleaned Main DF after Regex implementation for removing Special Characters and filtering out rows where Function Code Length >1

In [40]:
cleaned_DF = mainDF.loc[mainDF['Function'].apply(lambda x: len(x) == 1)]

In [41]:
cleaned_DF.head()

,Country,Location,Name,NameWoDiacritics,Subdivision,Status,Function,Date,IATA,Coordinates,Remarks
0,AU,ALO,Adelong,Adelong,NSW,RL,3,607.0,NaN,3519S 14804E,NaN
1,AU,ANB,Agnes Bank,Agnes Bank,NSW,RL,3,1501.0,NaN,3336S 15042E,NaN
2,AU,AGW,Agnew,Agnew,QLD,AI,4,9601.0,NaN,NaN,NaN
3,AU,WSY,Airlie Beach,Airlie Beach,QLD,AI,4,9912.0,NaN,NaN,NaN
4,AU,AIR,Airlie Terminal,Airlie Terminal,WA,RL,1,212.0,NaN,2120S 11510E,Airlie Island


In [42]:
cleaned_DF.shape

(84966, 11)

###### Merge all three  Cleaned DF, Function and Country Dataframe

In [43]:
import warnings
warnings.filterwarnings("ignore")

In [44]:
#Removing white spaces from Joining Columns 
#import warnings
#warnings.filterwarnings("ignore")
cleaned_DF['Country'] = cleaned_DF['Country'].str.strip()
cleaned_DF['Function'] = cleaned_DF['Function'].str.strip()
countryDF['CountryCode'] = countryDF['CountryCode'].str.strip()
functionDF['FunctionCode'] = functionDF['FunctionCode'].str.strip()

In [45]:
#coding for merged dataframe 
merged_df_1 = pd.merge(cleaned_DF,countryDF, left_on='Country',right_on ='CountryCode',how ='left')

In [46]:
merged_df_1.shape

(84966, 13)

In [47]:
merged_df_1.head()

,Country,Location,Name,NameWoDiacritics,Subdivision,Status,Function,Date,IATA,Coordinates,Remarks,CountryCode,CountryName
0,AU,ALO,Adelong,Adelong,NSW,RL,3,607.0,NaN,3519S 14804E,NaN,AU,Australia
1,AU,ANB,Agnes Bank,Agnes Bank,NSW,RL,3,1501.0,NaN,3336S 15042E,NaN,AU,Australia
2,AU,AGW,Agnew,Agnew,QLD,AI,4,9601.0,NaN,NaN,NaN,AU,Australia
3,AU,WSY,Airlie Beach,Airlie Beach,QLD,AI,4,9912.0,NaN,NaN,NaN,AU,Australia
4,AU,AIR,Airlie Terminal,Airlie Terminal,WA,RL,1,212.0,NaN,2120S 11510E,Airlie Island,AU,Australia


In [48]:
#coding for merged dataframe 
merged_final_df = pd.merge(merged_df_1,functionDF, left_on='Function',right_on ='FunctionCode',how ='left')

In [49]:
merged_final_df.head()

,Country,Location,Name,NameWoDiacritics,Subdivision,Status,Function,Date,IATA,Coordinates,Remarks,CountryCode,CountryName,FunctionCode,FunctionDescription
0,AU,ALO,Adelong,Adelong,NSW,RL,3,607.0,NaN,3519S 14804E,NaN,AU,Australia,3,Road Terminal
1,AU,ANB,Agnes Bank,Agnes Bank,NSW,RL,3,1501.0,NaN,3336S 15042E,NaN,AU,Australia,3,Road Terminal
2,AU,AGW,Agnew,Agnew,QLD,AI,4,9601.0,NaN,NaN,NaN,AU,Australia,4,Airport
3,AU,WSY,Airlie Beach,Airlie Beach,QLD,AI,4,9912.0,NaN,NaN,NaN,AU,Australia,4,Airport
4,AU,AIR,Airlie Terminal,Airlie Terminal,WA,RL,1,212.0,NaN,2120S 11510E,Airlie Island,AU,Australia,1,"Port, as defined in Rec 16"


In [50]:
merged_final_df.shape

(84966, 15)

In [51]:
merged_final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 84966 entries, 0 to 84965
Data columns (total 15 columns):
Country                84966 non-null object
Location               84966 non-null object
Name                   84966 non-null object
NameWoDiacritics       84966 non-null object
Subdivision            73433 non-null object
Status                 84962 non-null object
Function               84966 non-null object
Date                   84942 non-null float64
IATA                   351 non-null object
Coordinates            62642 non-null object
Remarks                697 non-null object
CountryCode            84966 non-null object
CountryName            84966 non-null object
FunctionCode           84966 non-null object
FunctionDescription    84966 non-null object
dtypes: float64(1), object(14)
memory usage: 10.4+ MB


Distribution of non Null Values looks appropriate in the final merged dataset

#### Splitting Lattitude and Longitude in to be processed separately and performing basic sanity checks

In [52]:
merged_final_df[['Lattitude','Longitude']] =  merged_final_df['Coordinates'].loc[merged_final_df['Coordinates'].str.split().str.len() == 2].str.split(expand=True)

In [53]:
# Checking for values with only alphabets
merged_final_df[merged_final_df['Lattitude'].str.contains('^0-9')==True]

,Country,Location,Name,NameWoDiacritics,Subdivision,Status,Function,Date,IATA,Coordinates,Remarks,CountryCode,CountryName,FunctionCode,FunctionDescription,Lattitude,Longitude


In [54]:
# Checking for values with only alphabets
merged_final_df[merged_final_df['Longitude'].str.contains('^0-9')==True]

,Country,Location,Name,NameWoDiacritics,Subdivision,Status,Function,Date,IATA,Coordinates,Remarks,CountryCode,CountryName,FunctionCode,FunctionDescription,Lattitude,Longitude


Coordinate Data seems to be bit erroneous as a lot of records don't follow the Degrees Minutes format apprpriately

#### Converting Coordinates in DDMM[N/S] DDMM[W/E] to Decimal Degree Format

Basic Idea on Lat/Long Notation
https://www.maptools.com/tutorials/lat_lon/formats

We have Lattitude/Longitude Data in DDMM[N/S] DDDMM[W/E] format and we need it in Degrees Decimal Format. We just need to convert the Minutes to degress and add to the deegress already present.

* 1° = 60 Mins

* 1 Min = (1/60)°

Also,Positive values of latitude are north of the equator, negative values to the south. Watch the sign on the longitude, most programs use negative values for west longitude, but a few are opposite.

I'll use this basic conversion by parsing the dataframe column and apply a function.


In [55]:
##This logic should be able to parse both  DDMM[N/S] & DDDMM[W/E] 

def DegMin2DegDeci(x):
    x = str(x)
    direction = None
    # extract degrees
    if len(x) == 5:
        direction = x[-1]
        mins = x[-3:-1]
        degrees = x[:2]
        degdec = float(degrees) + float(mins)/60
    elif len(x) == 6:
        direction = x[-1]
        mins = x[-3:-1]
        degrees = x[:3]
        degdec = float(degrees) + float(mins)/60
    else:
        degdec = None
    
    
    
    if direction in ('S','W'):
        degdec*= -1
    
    return degdec

In [56]:
DegMin2DegDeci('14804E')

148.06666666666666

In [57]:
## Testing DegMin2DegDeci('3519S') -35.31666666666667

In [58]:
## Testing DegMin2DegDeci('14804E') 148.06666666666666

In [59]:
merged_final_df['Lattitude'] = merged_final_df['Lattitude'].apply(lambda x: DegMin2DegDeci(x))


In [60]:
merged_final_df['Longitude'] = merged_final_df['Longitude'].apply(lambda x: DegMin2DegDeci(x))

In [61]:
final_DF = merged_final_df[['Country','CountryName','Location','Name','FunctionDescription','Longitude','Lattitude']]

In [62]:
# renaming Columns appropriately as expected
final_DF.rename(columns={'Country':'Country_code',
                         'CountryName':'country_name',
                         'Location':'Location_code',
                         'Name':'Location_Name',
                         'FunctionDescription':'Location_Type'
                        },inplace=True)

In [63]:
final_DF.head()

,Country_code,country_name,Location_code,Location_Name,Location_Type,Longitude,Lattitude
0,AU,Australia,ALO,Adelong,Road Terminal,148.066667,-35.316667
1,AU,Australia,ANB,Agnes Bank,Road Terminal,150.700000,-33.600000
2,AU,Australia,AGW,Agnew,Airport,NaN,NaN
3,AU,Australia,WSY,Airlie Beach,Airport,NaN,NaN
4,AU,Australia,AIR,Airlie Terminal,"Port, as defined in Rec 16",115.166667,-21.333333


In [64]:
final_DF.to_csv('geo_location.csv',sep = ',', index = False)